In [68]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import datetime

In [69]:
def get_user_tweets(username, num_tweets, start_date, end_date):
    tweets = []
    
    for tweet in sntwitter.TwitterSearchScraper(search(username=username, since=start_date, until=end_date, retweet="y", replies="y")).get_items():
        tweets.append(tweet)
        if len(tweets) == num_tweets:
            break
    
    return tweets


def get_tweet_comments(username, tweet_id, num_comments):
    comments = []
    for comment in sntwitter.TwitterSearchScraper(f"to:{username} conversation_id:{tweet_id}").get_items():
        comments.append([comment.date, comment.rawContent])
        if len(comments) == num_comments:
            break
    return comments


def search(text="",username="",since="",until="",retweet="",replies=""): 
    q = text 
    if username!='': 
        q += f" from:{username}"
    if until=='': 
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d') 
    q += f" until:{until}" 
    if since=='': 
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') - datetime.timedelta(days=7), '%Y-%m-%d') 
    q += f" since:{since}" 
    if retweet == 'y': 
        q += f" exclude:retweets" 
    if replies == 'y': 
        q += f" exclude:replies" 
    
    return q 

In [70]:
tweets = []

start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

current_date = start_date
while current_date <= end_date:
    current_date_str = current_date.strftime('%Y-%m-%d')
    tomorrow_str = (current_date + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    tweet = get_user_tweets('elonmusk', 1, current_date_str, tomorrow_str)
    if len(tweet) > 0:
        id = tweet[0].id
        comments = get_tweet_comments('elonmusk', id, 100)
        tweets.extend(comments)
    current_date += datetime.timedelta(days=1)

Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping after 20 empty pages
Stopping a

In [71]:
print(len(tweets))

42939


In [72]:
tweets_df = pd.DataFrame(tweets, columns=['date', 'content'])
tweets_df.head()

tweets_df.to_csv('elonmusk.csv', index=False)

In [83]:
df = pd.read_csv('elonmusk.csv')
df.shape

(42939, 2)

In [85]:
df = df[~df['content'].str.contains('@@@@@@@@')]
df = df[~df['content'].str.contains('http://')]
df = df[~df['content'].str.contains('https://')]
df = df.dropna()
df.shape

(37488, 2)